In [12]:
library('pROC')
library('IRdisplay')
library('lme4')
library('Hmisc')
library('MLmetrics')
library('ggExtra')

In [13]:
# run after Error Prediction Model - Data Preparation, where wordLevelChanges.csv is built
# Note that this notebook is in R, whereas data preparation was in Python

In [14]:
wdfs = read.csv('output/wordLevelChanges.csv', stringsAsFactors=F)
wdfs$change = sapply(wdfs$code, function(x){
    if (x == 'M'){
        return(0)
    } else { 
        return(1)
    }
})
wdfs$one =1

In [15]:
names(wdfs)

[1] "X"                                                  
  [2] "bnc_unigram_index"                                  
  [3] "bnc_unigram_prob"                                   
  [4] "bnc_unigram_word"                                   
  [5] "bnc_trigram_index"                                  
  [6] "bnc_trigram_prob"                                   
  [7] "bnc_trigram_word"                                   
  [8] "roark_scores_prefix"                                
  [9] "roark_scores_word"                                  
 [10] "roark_scores_prefix.1"                              
 [11] "roark_scores_srprsl"                                
 [12] "roark_scores_SynSp"                                 
 [13] "roark_scores_LexSp"                                 
 [14] "roark_scores_ambig"                                 
 [15] "roark_scores_open"                                  
 [16] "roark_scores_rernk"                                 
 [17] "roark_scores_toprr"                                 
 [18] "roark_scores_stps"                                  
 [19] "big_lm_scores_Unnamed..0"                           
 [20] "big_lm_scores_prob"                                 
 [21] "big_lm_scores_word"                                 
 [22] "kenlm_scores_prob"                                  
 [23] "kenlm_scores_preceding"                             
 [24] "kenlm_scores_unk"                                   
 [25] "kenlm_scores_words"                                 
 [26] "sCounter"                                           
 [27] "sentence_index"                                     
 [28] "word"                                               
 [29] "code"                                               
 [30] "input_subject"                                      
 [31] "output_subject"                                     
 [32] "conc_bigram"                                        
 [33] "conc_conc_m"                                        
 [34] "conc_conc_sd"                                       
 [35] "conc_unknown"                                       
 [36] "conc_total"                                         
 [37] "conc_percent_known"                                 
 [38] "conc_subtlex"                                       
 [39] "conc_dom_pos"                                       
 [40] "kpm_alternative_spelling"                           
 [41] "kpm_freq_pm"                                        
 [42] "kpm_dom_pos_subtlex"                                
 [43] "kpm_nletters"                                       
 [44] "kpm_nphon"                                          
 [45] "kpm_nsyll"                                          
 [46] "kpm_lemma_highest_pos"                              
 [47] "kpm_aoa_kup"                                        
 [48] "kpm_perc_known"                                     
 [49] "kpm_aoa_kup_lem"                                    
 [50] "kpm_perc_known_lem"                                 
 [51] "kpm_aoa_bird_lem"                                   
 [52] "kpm_aoa_bristol_lem"                                
 [53] "kpm_aoa_cort_lem"                                   
 [54] "kpm_aoa_schock"                                     
 [55] "pic_ipa"                                            
 [56] "pic_ipa_ss_array"                                   
 [57] "pic_ipa_ss"                                         
 [58] "pic_ipa_n"                                          
 [59] "pic_ortho"                                          
 [60] "pic_ortho_ss_array"                                 
 [61] "pic_ortho_ss"                                       
 [62] "pic_ortho_n"                                        
 [63] "pic_character"                                      
 [64] "pic_character_ss_array"                             
 [65] "pic_character_ss"                                   
 [66] "pic_character_n"                                    
 [67] "lic_mean_surprisal_weighted"       

In [16]:
# convert log probabilities to surprisals
wdfs$bnc_unigram_prob = -1 * wdfs$bnc_unigram_prob
wdfs$bnc_trigram_prob = -1 * wdfs$bnc_trigram_prob
wdfs$kenlm_scores_prob = -1 * wdfs$kenlm_scores_prob
wdfs$big_lm_scores_prob = -1 * wdfs$big_lm_scores_prob
wdfs$gpt2_normal_scores_prob = -1 * wdfs$gpt2_normal_scores_prob
wdfs$gpt2_medium_scores_prob = -1 * wdfs$gpt2_medium_scores_prob
wdfs$bert_scores_prob = -1 * wdfs$bert_scores_prob
wdfs$bart_scores_prob = -1 * wdfs$bart_scores_prob

ERROR: Error in `$<-.data.frame`(`*tmp*`, gpt2_scores_prob, value = numeric(0)): replacement has 0 rows, data has 27290


In [ ]:
wdfs[1,c('bnc_unigram_prob','bnc_trigram_prob','kenlm_scores_prob','big_lm_scores_prob',
        'gpt2_normal_scores_prob', 'gpt2_medium_scores_prob', 'bert_scores_prob', 'bart_scores_prob')]

In [ ]:
dim(wdfs)

[ ] part of speech  
[X] number of syllables: kpm_nsyll  
[X] word length: kpm_nphon  
[X] aoa_rating: kpm_aoa_kup  
[X] concreteness: conc_conc_m  
[X] phonological neighborhood density: pld20  
[X] contextual diversity: Need Subtlexus: SUBTLCD  

random effects structure
https://www4.stat.ncsu.edu/~reich/ABA/code/DICmixed

In [ ]:
wdfs

In [ ]:
# interpolate missing values in the input

# many of these entries are related to Table 4
fixed_effects = c('one','word','bnc_unigram_prob', 'bnc_trigram_prob', 'roark_scores_SynSp', 
'roark_scores_LexSp', 'big_lm_scores_prob', 'kenlm_scores_prob','sCounter', 
'kpm_aoa_kup','kpm_nphon', 'kpm_nsyll','conc_conc_m','pld20', 'SUBTLCD', 
'normalized_biglm_probability', 'normalized_WSJ_Roark_Negative.Log.Probability',
'normalized_BNC_KNN_unigramProb', 'normalized_BNC_KNN_trigramProb',
'normalized_kenlm_probability', 'normalized_bllip_probability', 
'normalized_bllip_wsj_probability', 'normalized_mikolov_wsj_probability','pic_ipa_ss','kenlm_scores_prob',
                 'normalized_gpt2_normal_probability', 'normalized_gpt2_medium_probability', 'normalized_bert_probability',
                 'normalized_bart_probability')

for(fixed_effect in fixed_effects){
    print(paste(fixed_effect, 'missing', length(which(is.na(wdfs[,fixed_effect]))), 'observations'))
  wdfs[is.na(wdfs[,fixed_effect]), fixed_effect] <- mean(wdfs[,fixed_effect], na.rm = TRUE)
}

# Fixed Effects Logistic Regression

In [ ]:
fixed_effects

In [ ]:
# basic fixed effects logistic regression

wdf = wdfs[,fixed_effects]
wdf$changed = as.numeric(wdfs$code != 'M')
model <- glm(changed ~ bnc_unigram_prob + bnc_trigram_prob + roark_scores_LexSp +  roark_scores_SynSp
+ big_lm_scores_prob + bert_scores_prob + gpt2_normal_scores_prob + gpt2_medium_scores_prob
             + bart_scores_prob
             + sCounter + kpm_aoa_kup + kpm_nphon + kpm_nsyll + conc_conc_m
, family=binomial(link='logit'),data=wdf)

In [ ]:
summary(model)

In [ ]:
wdfs$predicted = predict(model, wdf, type='response')
df = data.frame(prob=wdfs$predicted, change=wdfs$change)
g <- roc(change ~prob, data = df)
plot(g)    

In [ ]:
auc(g)

In [ ]:
colorizeVector = function(vector){
    colorramp = colorRampPalette(c("green", "red"))(n = 101)
    quantiles = quantile(vector, probs=seq(0,1,.01), na.rm=T)
    percentile_limits = sapply(vector, function(x){
        which.max(x <= quantiles)	
    })
    percentile = sapply(percentile_limits, function(x){ifelse(length(x) > 0, x, NA)})
    colors = colorramp[percentile]
    return(colors)
}
wdfs$color = colorizeVector(wdfs$predicted)

colorizeSentence = function(df, sentenceIndex, format = 'HTML'){
    sentence = subset(df, sentence_index == sentenceIndex)
    sentence = sentence[order(sentence$bnc_unigram_index),]
    if (format =='HTML'){
        html_pieces = paste0('<font color="',sentence$color,'">',sentence$word, '</font>')    
        return(paste(html_pieces, collapse =' '))
    } else if (format == 'latex'){
        latex_pieces = paste0('{color[HTML]{',gsub('#','',sentence$color),'}',sentence$word,'}')
        #{\color[HTML]{color}text}
        return(paste(latex_pieces, collapse =' '))
    }
}

colorizeSentence(wdfs,1661,'latex')
display_html(colorizeSentence(wdfs,1661))

# Mixed Effects Logistic Regression Model

In [ ]:
print(names(wdf))
print(nrow(wdf))

In [ ]:
# add residuals 
wdf$resid_bnc_trigram_prob = lm(bnc_trigram_prob ~ bnc_unigram_prob, data=wdf)$residuals

# Note to self: p. 33 of the paper.
for (structuredModel in c('roark_scores_SynSp','big_lm_scores_prob','kenlm_scores_prob',
                          'gpt2_normal_scores_prob', 'gpt2_medium_scores_prob', 'bert_scores_prob',
                         'bart_scores_prob')){
    wdf[[paste0('resid_',structuredModel)]] = lm(as.formula(paste0(structuredModel, 
        ' ~ bnc_trigram_prob + bnc_unigram_prob')), data=wdf)$residuals
}

In [ ]:
wdf$subject_id = as.factor(wdfs$user)
wdf$upstream_subject_id = as.factor(wdfs$upstream_subject_id)

mixed_model <- glmer(changed ~ bnc_unigram_prob + resid_bnc_trigram_prob + resid_roark_scores_SynSp 
+ resid_big_lm_scores_prob + resid_kenlm_scores_prob
                     + resid_gpt2_normal_scores_prob + resid_gpt2_medium_scores_prob + resid_bert_scores_prob
                     + resid_bart_scores_prob
                     + sCounter + kpm_aoa_kup + kpm_nphon + kpm_nsyll + conc_conc_m + pld20 
+ (1|subject_id) + (1|upstream_subject_id)
,family=binomial(link='logit'),data=wdf, control=glmerControl(optimizer="bobyqa",
                            optCtrl=list(maxfun=2e5)))

In [ ]:
# how useful is knowing the probability of the sentence -- takes forever to run
#+ normalized_biglm_probability + normalized_WSJ_Roark_Negative.Log.Probability 
#+ normalized_BNC_KNN_unigramProb + normalized_BNC_KNN_trigramProb 
#+ normalized_kenlm_probability + normalized_bllip_probability
#+ normalized_bllip_wsj_probability + normalized_mikolov_wsj_probability        

In [11]:
# This is Table 4

summary(mixed_model)

ERROR: Error in summary(mixed_model): object 'mixed_model' not found


In [ ]:
wdfs$me_predicted = predict(mixed_model, wdf, type='response')
initial_sentences = subset(wdfs, upstream_pointer ==-1 & global_chain ==0)

colHTMLStore = mat.or.vec(20,1)
for (sid in c(0:39)){
    target_index = unique(subset(initial_sentences, stimulus_id == sid)$sentence_index)
    
    if (sid <= 20){
         colHTMLStore[sid] = colorizeSentence(wdfs, target_index, format='latex')   
    }
    colorHTML = colorizeSentence(wdfs,target_index)
    display_html(colorHTML)
}

write(paste0(colHTMLStore, collapse='\n \\\\ \n'), file = "figures/logisticRegression.txt")

In [ ]:
wdfs$me_predicted = predict(mixed_model, wdf, type='response')
df = data.frame(prob=wdfs$me_predicted, change=wdfs$change)
g <- roc(change ~prob, data = df)
print(plot(g))
auc(g)

In [ ]:
mixed_model_nolms <- glmer(changed ~ sCounter + kpm_aoa_kup + kpm_nphon + kpm_nsyll + conc_conc_m + pld20 
+ (1|subject_id) + (1|upstream_subject_id)
,family=binomial(link='logit'),data=wdf, control=glmerControl(optimizer="bobyqa",
                            optCtrl=list(maxfun=2e5)))

In [ ]:
wdfs$nolms_predicted = predict(mixed_model_nolms, wdf, type='response')
df = data.frame(prob=wdfs$nolms_predicted, change=wdfs$change)
g <- roc(change ~prob, data = df)
print(plot(g))
auc(g)

In [ ]:
# output the graph
source('telephone_analysis.R')

replacements = list()
replacements[['bnc_unigram_prob']] = 'BNC unigram surprisal'
replacements[['resid_bnc_trigram_prob']] = 'Residualized BNC trigram surprisal'
replacements[['resid_roark_scores_SynSp']] = 'Residualized Roark PCFG syntactic surprisal'
replacements[['resid_big_lm_scores_prob']] = 'Residualized Big LM surprisal'
replacements[['resid_kenlm_scores_prob']] = 'Residualized DS 5-gram surprisal'
replacements[['resid_gpt2_normal_scores_prob']] = 'Residualized GPT-2 surprisal'
replacements[['resid_gpt2_medium_scores_prob']] = 'Residualized GPT-2 medium surprisal'
replacements[['resid_bert_scores_prob']] = 'Residualized BERT surprisal'
replacements[['resid_bart_scores_prob']] = 'Residualized BART surprisal'
replacements[['sCounter']] = 'Position in sentence'
replacements[['kpm_aoa_kup']] = 'Age of acquisition'
replacements[['kpm_nphon']] = 'Number of phonemes'
replacements[['kpm_nsyll']] = 'Number of syllables'
replacements[['conc_conc_m']] = 'Concreteness'
replacements[['pld20']] = 'Phonological Neighborhood Density (PLD20)'
replacements[['upstream_subject_id']] = 'Speaker ID'
replacements[['subject_id']] = 'Listener ID'

modelName = "wordLevel"
wordLevelCaption = 'Mixed-effects logistic regression predicting whether a word will be transmitted successfully on the basis of its surprisal under various language models as well as other word properties. Significance of fixed-effects is computed following \\citet{satterthwaite1946}.'
wordLevelLabel = 'tab:wordlevel_lm'

modelToTable('mixed_logistic', modelName, mixed_model, replacements=replacements,file=paste0('LMs/',modelName,'_lm.tex'), 
        printVars = T, caption = wordLevelCaption, label = wordLevelLabel, where= 't')


Cross-validation 

In [ ]:
# propotion matches per user

In [ ]:
(wdfs$code)

In [ ]:
names(wdfs)

# Flag Behavior vs. Edit Behavior

In [ ]:
propMatch = aggregate(code ~ output_subject, wdfs, function(x){sum(x == 'M') / length(x)})
names(propMatch) = c('user','propMatch')
numWords =  aggregate(code ~ output_subject, wdfs, function(x){numWords = length(x)})
names(numWords)  = c('user', 'numWords')
numSentences = aggregate(stimulus_id ~ input_subject, wdfs, function(x){length(unique(x))})
names(numSentences) = c('user','numSentencesNotFlagged')
wdfs_users = merge(merge(propMatch, numWords), numSentences)

In [ ]:
flag_behavior = rbind(
    read.csv('output/180419_AMT_lengthLimitedGPU_flags.csv',
    stringsAsFactors=F),
    read.csv('output/180624_AMT_lengthLimitedGPU_flags.csv',
    stringsAsFactors=F)
)    

In [ ]:
stim_flags = subset(flag_behavior, flag_type == 'stimulus')

In [ ]:
flag_reasons = data.frame(table(stim_flags$reason))
flag_reasons = flag_reasons[order(flag_reasons$Freq, decreasing=T),]

In [ ]:
flags_per_user  = aggregate(reason ~ user, stim_flags, length)
names(flags_per_user) = c('user','numFlags')

In [ ]:
wdfs_users_with_flags = merge(wdfs_users, flags_per_user, all.x=T)
#add back users with 0-counts
wdfs_users_with_flags$numFlags[is.na(wdfs_users_with_flags$numFlags)] = 0
wdfs_users_with_flags$propFlags = wdfs_users_with_flags$numFlags / 
(wdfs_users_with_flags$numFlags + wdfs_users_with_flags$numSentencesNotFlagged)

In [ ]:
numFlagsAndUnflagged = sum(wdfs_users_with_flags$numSentencesNotFlagged
                          ) + sum(wdfs_users_with_flags$numFlags)

In [ ]:
# Proportion of inputs from other participants which were flagged
sum(wdfs_users_with_flags$numFlags)  / numFlagsAndUnflagged

In [ ]:
flag_reasons$propOfInputs = flag_reasons$Freq / numFlagsAndUnflagged
flag_reasons$propOfFlags = flag_reasons$Freq / sum(flag_reasons$Freq)
head(flag_reasons, n=10)

In [ ]:
lm1 = lm((1 - propMatch) ~ numFlags, subset(wdfs_users_with_flags, numWords > 25))
m = summary(lm1)
print(m)

In [ ]:
eq <- substitute(italic(y) == a + b %.% italic(x)*","~~italic(R)^2~"="~r2~","~~italic(p)~"="~pval, 
         list(a = format(coef(m)[1], digits = 2), 
              b = format(coef(m)[2], digits = 2), 
             r2 = round(m$adj.r.squared, 3),
            pval = unname(round(pf(m$fstatistic[1], m$fstatistic[2], m$fstatistic[3],
      lower.tail = FALSE), 3))
             ))

eq = as.character(as.expression(eq))               
eq

In [ ]:
p1= ggplot(subset(wdfs_users_with_flags, numWords > 25)) + geom_point(aes(x=propFlags, y= 1 - propMatch)
) + geom_smooth(aes(x=propFlags, y=1 - propMatch), method='gam') + ylab(
'Proportion of Words Changed By Partcipant\n(Participant Accepted Sentences)') + xlab(
'Proportion of Input Sentences Flagged by Participant\n(Participant Rejected Sentences)'
) + theme_classic() + annotate("text", x = .4, y = .4, label = eq, parse = TRUE, 
    color='red', size=5)
p1 = ggMarginal(p1, type="histogram")
print(p1)

In [ ]:
ggsave( 'figures/wordChangesVsFlags.pdf', plot= p1, width=6, height=6)
ggsave( 'figures/wordChangesVsFlags.png', plot= p1, width=6, height=6)